# Model Output

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet18


In [2]:
model = resnet18().eval()

In [6]:
list(model.modules())[-1]

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
out = model(torch.randn(1, 3, 32, 32))

In [ ]:
out.shape

In [ ]:
np.max(out.detach().numpy()), np.min(out.detach().numpy())

In [ ]:
plt.figure(figsize=(30, 10))
plt.bar(range(1000), out.detach().numpy().reshape(-1))
plt.show()

# Configuration

In [ ]:
import json
from KD_Lib.models.resnet import ResNet18, ResNet50, ResNet152

In [ ]:
class Cfg:
    def __init__(self, dict=None):
        if dict is not None:
            for key in dict:
                setattr(self, key, dict[key])
            return
        
        self.MODE: str = 'shake' # 'kd' or 'dml' or 'shake'
        self.DATASET: str = 'cifar100' # 'cifar10' or 'cifar100'
        self.CLASSES: int = 100
        self.DATA_PATH: str = '../Knowledge-Distillation-Zoo/datasets/'
        self.BATCH_SIZE: int = 128
        self.TEACHER = 'resnet152' 
        self.STUDENT = 'resnet18'
        self.LR: float = 0.1
        self.LR_MIN: float = 1e-6 #1e-5
        self.T: float = 1.0
        self.W: float = 0.5
        self.EPOCHS: int = 200
        self.SCHEDULER: str = 'cos' # 'cos' or 'step'
        self.TEACHER_WEIGHTS: str = f'./models/teacher_{self.DATASET}_{self.MODE}.pt'
        self.PARALLEL: bool = False
        self.EXP: str = f"{self.MODE}_{self.DATASET}"

In [ ]:
cfg = Cfg()
cfg.__dict__

In [ ]:
with open("cfg.json", "w") as file:
    json.dump(cfg.__dict__, file)
 
with open("cfg.json", "r") as file:
    loaded_cfg = json.load(file)
 
print(loaded_cfg)

In [ ]:
cfg = Cfg(loaded_cfg)

In [ ]:
cfg.__dict__

# Visualize Scheduler

In [ ]:
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, MultiStepLR
import matplotlib.pyplot as plt

In [ ]:
STEPS = 200
LR = 0.1
ETA = 1e-5

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = CosineAnnealingLR(optimizer, STEPS, eta_min=ETA, last_epoch=-1)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = LinearLR(optimizer, total_iters=STEPS, start_factor=1, end_factor=ETA/LR)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

lrs = []
optimizer = torch.optim.SGD([torch.tensor(1)], lr=LR)
scheduler = MultiStepLR(optimizer, [60, 120, 180], gamma=0.1)
for _ in range(STEPS):
    optimizer.step()
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
plt.plot(lrs, label=scheduler.__class__.__name__)

#plt.semilogy()
plt.legend()
plt.show()

## CUB200

In [ ]:
import torch
import numpy as np
from torchvision import datasets, transforms
from datasets import Cub200

DATASET = 'cub200'
DATA_PATH = '../Knowledge-Distillation-Zoo/datasets/'
BATCH_SIZE = 128

In [ ]:
if DATASET == 'cifar100':
    dataset = datasets.CIFAR100
    mean = (0.5071, 0.4865, 0.4409)
    std  = (0.2673, 0.2564, 0.2762)
    imsize = 32
elif DATASET == 'cub200':
    dataset = Cub200
    mean = (104/255.0, 117/255.0, 128/255.0)
    std = (1/255.0, 1/255.0, 1/255.0)
    imsize = 227

train_transform = transforms.Compose([
    transforms.RandomCrop(imsize, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])

trainset = dataset(root=DATA_PATH, train=True, download=False, transform=train_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
trainset[0][0].shape

In [ ]:
trainset[0][0].shape

In [ ]:
mean

## ResNet

In [ ]:
from torchvision.models import resnet18 as ResNet18, resnet50 as ResNet50, resnet152 as ResNet152

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
model = ResNet18(weights=None, num_classes=100)

In [ ]:
get_n_params(model)

In [ ]:
from KD_Lib.models.resnet import ResNet18, ResNet50, ResNet152

In [ ]:
model_new = ResNet18(num_classes=100)

In [ ]:
len(list(model_new.parameters()))

In [ ]:
get_n_params(model_new)

In [ ]:
model

In [ ]:
model_new

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from KD_Lib.models.resnet_torch import get_ResNet

In [ ]:
model = get_ResNet('resnet18', 10).eval()

In [ ]:
test = torch.randn(1, 3, 32, 32)

In [ ]:
model.forward(test, norm_feats=False)

In [ ]:
np.hist(model.forward(test, norm_feats=True))

In [ ]:
out, feats, weight, bias = model.forward(torch.randn(2, 3, 32, 32), return_feats=True)